# Language Server Protocol (LSP) Integration Example

The [Language Server Protocol](https://microsoft.github.io/language-server-protocol/) is an open-standard protocol for communication between development tools and Language Servers.

Specifically, [Language Server for Kotlin](https://github.com/Kotlin/kotlin-lsp) provides features that can integrate into JetBrains IDEs for Kotlin development.

## Downloading the Language Server for Kotlin

The [Language Server for Kotlin](https://github.com/Kotlin/kotlin-lsp) is available as a standalone distribution.
Please follow the instructions on the project page to download the latest version for your operating system.
Make sure that the `kotlin-lsp` executable is available in your `PATH` environment variable.

## IntelliJ Platform Integration and Bundled Plugin

Declare an integration with IntelliJ Platform that will configure necessary libraries.

In [ ]:
%use intellij-platform

The Language Server Protocol support is available in the commercial JetBrains IDEs.
To enable this, load a bundled plugin that provides this functionality.
Use `loadBundledPlugins` function to load it.

In [ ]:
loadBundledPlugins("com.intellij.modules.ultimate")

## Integrating the Server with Kotlin Files

To integrate the Kotlin LSP, two components are required:

1. an LSP server descriptor that indicates supported files. Additionally, it provides a command line to start the server in a standalone process.
2. an LSP support provider that will ensure that the server is available for a given file.

### Server Descriptor

Create a server descriptor that will support files with `.kn` extension.
To start the LSP server, a command line needs to be created. Use the `kotlin-lsp` binary available in the `PATH` environment variable with `--stdio` switch that enables the communication via standard input and output streams.

In [ ]:
import com.intellij.execution.configurations.GeneralCommandLine
import com.intellij.openapi.project.Project
import com.intellij.openapi.vfs.VirtualFile
import com.intellij.platform.lsp.api.ProjectWideLspServerDescriptor

class KotlinLspServerDescriptor(project: Project) : ProjectWideLspServerDescriptor(project, "KN Kotlin LSP") {
    override fun isSupportedFile(file: VirtualFile) = file.extension == "kt"

    override fun createCommandLine() = GeneralCommandLine("kotlin-lsp", "--stdio")
}

## LSP Support Provider

Create a LSP Support provider that will start the LSP server, if necessary, with the corresponding Server Descriptor.

In [ ]:
import com.intellij.platform.lsp.api.LspServerSupportProvider

class KotlinLspServerSupportProvider : LspServerSupportProvider {
    override fun fileOpened(project: Project, file: VirtualFile, serverStarter: LspServerSupportProvider.LspServerStarter) {
        if (file.extension == "kt") {
            serverStarter.ensureServerStarted(KotlinLspServerDescriptor(project))
        }
    }
}

## Registering an Extension Point

The `KotlinLspServerSupportProvider` is an extension point that needs to be registered to the corresponding extension point.
We need to represent the plugin descriptor extension declaration in Kotlin code.

The plugin descriptor (`plugin.xml`) declaration in the full-fledged plugin is as follows:

```xml
<extensions defaultExtensionNs="com.intellij">
    <platform.lsp.serverSupportProvider implementation="com.example.KotlinLspServerSupportProvider"/>
</extensions>
```

Luckily, such XML descriptor is unnecessary in this Kotlin Notebook.
Instead, create an instance of the LSP Server Support provider. Then, register it under corresponding extension point.

In [ ]:
import com.intellij.platform.lsp.api.LspServerManager
import com.intellij.platform.lsp.api.LspServerSupportProvider

runInEdt {
    registerExtension(LspServerSupportProvider.EP_NAME, KotlinLspServerSupportProvider())
}

## Testing the Integration

To test the integration, create a Kotlin file with some unnecessary constructs.

```kotlin
import java.util.List

class Hello {
    fun sayHello() = "Hello, world!"
}
```

Observe two warnings: an unnecessary import and an implicit return type for `sayHello` function.

[Show Intention Actions](https://www.jetbrains.com/help/idea/intention-actions.html) and observe two actions in the list provided by the LSP server.

* *Organize Imports* removes the unnecessary `import java.util.List`
* *Specify type explicitly* adds an explicit `String` return type to `sayHello` function.

These actions might be duplicated by native intention actions provided by the JetBrains IDEs such as *Organize imports* or *Specify return type explicitly*.

## Summary

- Load the `com.intellij.modules.ultimate` plugin to support the LSP functionality.
- Register a `LspServerSupportProvider` subclass into the `com.intellij.platform.lsp.serverSupportProvider` extension point.
- Create a `ProjectWideLspServerDescriptor` subclass to launch the LSP server in a standalone process and standard input and output streams as a communication channel.
- Observe how the functionality is provided in the Intention Action list.